<a href="https://colab.research.google.com/github/ahmtox/swept-plane-scanner/blob/main/computeShadowPlanes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HELLO ITS THIS ONE

In [ ]:
#@title Jerry

import numpy as np
import cv2

X = np.array([
    [
        [-1, -1],
        [-1, -1],
        [30, 15],
        [32, 1],
        [-1, -1],
        [1, 1],
        [30, 15],
        [-1, -1]
    ],
    [
        [0, 0],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
    ]
])

print(X[0])
print(X[1])

f1 = './mask_higher.jpg'
f2 = './mask_lower.jpg'
mask_high = cv2.imread(f1)
mask_low = cv2.imread(f2)

def computeShadowPlanes(mask_higher, mask_lower, bg_edge_points):
  print(bg_edge_points.shape[0])
  shadow_planes = np.zeros((bg_edge_points.shape[0],4))

  for i, frame in enumerate(bg_edge_points):
    print(frame)
    not_neg_ones = ~np.all(frame == [-1, -1], axis=1)

    print(not_neg_ones)

    # Find the index of the first True in not_neg_ones
    start_index = np.argmax(not_neg_ones)

    print(start_index)

    # From the start_index, find the next False in not_neg_ones, which indicates the end of the top section of non -1 points
    end_index = np.argmax(not_neg_ones[start_index:] == False) + start_index

    # If end_index is equal to start_index, it means no False found, extract till the end
    if end_index == start_index:
        end_index = len(frame)

    # Extract the top section of non -1 points
    top_section = frame[start_index:end_index]

    A1 = in2hom(top_section)

    # singular value decomposition
    U1,D1,VT1 = np.linalg.svd(A1)

    # best-fit line is the last row of V-transpose (i.e., last column of V)
    l1 = VT1[-1,]

    not_neg_ones = ~np.all(frame == [-1 -1], axis=1)

    # Find the index of the last True in not_neg_ones (reverse search)
    end_index = len(frame) - np.argmax(not_neg_ones[::-1]) - 1

    # From the end_index, find the next False in not_neg_ones when searching in reverse, indicating the start of the bottom section
    start_index = len(frame) - np.argmax((not_neg_ones[:end_index+1][::-1] == False)) - 1

    # If start_index is equal to end_index, it means no False found, start from the beginning
    if start_index == end_index:
        start_index = 0

    # Extract the bottom section of non -1 points
    bottom_section = frame[start_index:end_index+1]

    A2 = in2hom(bottom_section)

    # singular value decomposition
    U2,D2,VT2 = np.linalg.svd(A2)

    # best-fit line is the last row of V-transpose (i.e., last column of V)
    l2 = VT2[-1,]


    # Calculate a normal vector
    N = np.cross(l1, l2)

    # Get a point from one of the arrays (we use top_section here)
    known_point = top_section[0]

    # Find the resulting plane
    plane = np.dot(N, point)
    print("p", plane)
    shadow_planes[i] = plane

  return shadow_planes


In [39]:
import numpy as np
import cv2

def computeShadowPlanes(mask_higher, mask_lower, bg_edge_points):

  not_neg_ones = ~np.all(bg_edge_points == [-1, -1], axis=1)

  # Find the index of the first True in not_neg_ones
  start_index = np.argmax(not_neg_ones)

  # From the start_index, find the next False in not_neg_ones, which indicates the end of the top section of non -1 points
  end_index = np.argmax(not_neg_ones[start_index:] == False) + start_index

  # If end_index is equal to start_index, it means no False found, extract till the end
  if end_index == start_index:
      end_index = len(bg_edge_points)

  # Extract the top section of non -1 points
  top_section = bg_edge_points[start_index:end_index]

  A1 = in2hom(top_section)

  # singular value decomposition
  U1,D1,VT1 = np.linalg.svd(A1)

  # best-fit line is the last row of V-transpose (i.e., last column of V)
  l1 = VT1[-1,]

  not_neg_ones = ~np.all(bg_edge_points == [-1, -1], axis=1)

  # Find the index of the last True in not_neg_ones (reverse search)
  end_index = len(bg_edge_points) - np.argmax(not_neg_ones[::-1]) - 1

  # From the end_index, find the next False in not_neg_ones when searching in reverse, indicating the start of the bottom section
  start_index = len(bg_edge_points) - np.argmax((not_neg_ones[:end_index+1][::-1] == False)) - 1

  # If start_index is equal to end_index, it means no False found, start from the beginning
  if start_index == end_index:
      start_index = 0

  # Extract the bottom section of non -1 points
  bottom_section = bg_edge_points[start_index:end_index+1]

  A2 = in2hom(bottom_section)

  # singular value decomposition
  U2,D2,VT2 = np.linalg.svd(A2)

  # best-fit line is the last row of V-transpose (i.e., last column of V)
  l2 = VT2[-1,]


  # Calculate a normal vector
  N = np.cross(l1, l2)

  # Get a point from one of the arrays (we use top_section here)
  point = top_section[0]

  # Find the resulting plane
  shadow_plane = np.dot(N, point)

  return shadow_plane


In [ ]:
import numpy as np
import cv2

def computeShadowPlanes(foreground_mask, background_mask, front_edge_times, plane_h, plane_v, calMatrix, distCoeffs):
    """Compute shadow planes from video frames based on foreground and background masks."""

    def preprocess_points(mask, frame_number):
        """Retrieve and undistort edge pixels for a given frame and mask."""
        edge_pixels = np.argwhere((front_edge_times[:, :, frame_number] > 0) & mask)
        return cv2.undistortPoints(np.expand_dims(edge_pixels, axis=1), calMatrix, distCoeffs, None, calMatrix).squeeze()

    def compute_intersections(plane, points):
        """Compute intersections of rays from undistorted points with a plane."""
        intersections = np.empty((points.shape[0], 3))
        plane_normal = plane[:3]
        plane_d = plane[3]

        for i, point in enumerate(np.hstack((points, np.ones((points.shape[0], 1))))):
            ray_direction = np.linalg.inv(calMatrix).dot(point)
            scale_factor = -plane_d / np.dot(plane_normal, ray_direction)
            intersections[i] = scale_factor * ray_direction

        return intersections

    def fit_plane(points):
        """Fit a plane to a collection of points using Singular Value Decomposition."""
        centroid = points.mean(axis=0)
        svd_result = np.linalg.svd(points - centroid, full_matrices=False)
        plane_normal = svd_result[2][-1, :]
        return np.append(plane_normal, -np.dot(plane_normal, centroid))

    video_frames = front_edge_times.shape[2]
    shadow_planes = [fit_plane(np.vstack([
        compute_intersections(plane_h, preprocess_points(foreground_mask, frame)),
        compute_intersections(plane_v, preprocess_points(background_mask, frame))
    ])) for frame in range(video_frames)]

    return shadow_planes

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (<ipython-input-6-7249fd6f18e8>, line 40)

THIS IS FROM CHAT GPT THE FIRST ONE IS MINE

In [ ]:
import numpy as np
import cv2

def computeShadowPlanes(foreground_mask, background_mask, front_edge_times, plane_h, plane_v, calMatrix, distCoeffs):
    """Compute shadow planes from video frames based on foreground and background masks."""

    # Check if the front_edge_times array has an additional dimension for times
    if front_edge_times.ndim == 3 and front_edge_times.shape[-1] == 2:
        # Handle the case where we have start and end times
        edge_detected = np.any(front_edge_times > 0, axis=-1)
    else:
        # Handle the case where we only have one timestamp per pixel
        edge_detected = front_edge_times > 0

    def preprocess_points(mask, frame_number):
        """Retrieve and undistort edge pixels for a given frame and mask."""
        # Use the updated edge_detected array
        edge_pixels = np.argwhere(edge_detected[:, :, frame_number] & mask)
        return cv2.undistortPoints(np.expand_dims(edge_pixels, axis=1), calMatrix, distCoeffs, None, calMatrix).squeeze()

    def compute_intersections(plane, points):
        """Compute intersections of rays from undistorted points with a plane."""
        intersections = np.empty((points.shape[0], 3))
        plane_normal = plane[:3]
        plane_d = plane[3]

        for i, point in enumerate(np.hstack((points, np.ones((points.shape[0], 1))))):
            ray_direction = np.linalg.inv(calMatrix).dot(point)
            scale_factor = -plane_d / np.dot(plane_normal, ray_direction)
            intersections[i] = scale_factor * ray_direction

        return intersections

    def fit_plane(points):
        """Fit a plane to a collection of points using Singular Value Decomposition."""
        centroid = points.mean(axis=0)
        svd_result = np.linalg.svd(points - centroid, full_matrices=False)
        plane_normal = svd_result[2][-1, :]
        return np.append(plane_normal, -np.dot(plane_normal, centroid))

    video_frames = edge_detected.shape[2]
    shadow_planes = []

    for frame in range(video_frames):
        # Process points from the foreground and background separately to compute their shadow planes
        fg_points = preprocess_points(foreground_mask, frame)
        bg_points = preprocess_points(background_mask, frame)

        if fg_points.size > 0:
            intersections_h = compute_intersections(plane_h, fg_points)
        else:
            intersections_h = np.empty((0, 3))

        if bg_points.size > 0:
            intersections_v = compute_intersections(plane_v, bg_points)
        else:
            intersections_v = np.empty((0, 3))

        # Combine intersections only if there are any points
        if intersections_h.size > 0 or intersections_v.size > 0:
            all_intersections = np.vstack([intersections_h, intersections_v])
            shadow_plane = fit_plane(all_intersections)
            shadow_planes.append(shadow_plane)
        else:
            # Handle the case where there are no intersections
            shadow_planes.append(np.array([0, 0, 0, 0]))  # Placeholder for no shadow plane

    return shadow_planes

In [ ]:
import numpy as np
import cv2

def back_project_to_plane(plane, image_points, calMatrix, distCoeffs):
    # Convert to normalized image coordinates
    normalized = cv2.undistortPoints(np.expand_dims(image_points, axis=1), calMatrix, distCoeffs, None, calMatrix).squeeze()

    # Convert to homogenous coordinates by appending a 1
    homogenous = np.hstack([normalized, np.ones((normalized.shape[0], 1))])

    # Initialize array for 3D points
    points_3d = np.zeros((homogenous.shape[0], 3))

    # Plane coefficients
    a, b, c, d = plane

    for i, point in enumerate(homogenous):
        # Calculate scale for projection onto the plane
        scale = -d / (a * point[0] + b * point[1] + c)

        # Calculate 3D points
        x_3d, y_3d = point[:2] * scale
        z_3d = scale

        # Store the 3D point
        points_3d[i] = [x_3d, y_3d, z_3d]

    return points_3d

def compute_best_shadow_plane(points_h, points_v):
    # Combine points from both planes
    all_points = np.vstack([points_h, points_v])

    # Calculate the centroid of all points
    centroid = np.mean(all_points, axis=0)

    # Calculate the covariance matrix
    cov_matrix = np.cov((all_points - centroid).T)

    # Calculate the eigenvectors and eigenvalues of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # The eigenvector with the smallest eigenvalue will be the normal to the plane
    normal_vector = eigenvectors[:, np.argmin(eigenvalues)]

    # Calculate the plane coefficient d using the normal vector and centroid
    d = -np.dot(normal_vector, centroid)

    # Create the plane coefficients
    shadow_plane = np.append(normal_vector, d)

    return shadow_plane

def computeShadowPlanes(foreground_mask, background_mask, front_edge_times, plane_h, plane_v, calMatrix, distCoeffs):
    shadow_planes = []

    # Assuming front_edge_times is an HxWxT array, where T is the number of frames
    num_frames = front_edge_times.shape[2]

    for frame_idx in range(num_frames):
        # Get the edge pixels for the current frame
        edge_pixels = np.argwhere(front_edge_times[:, :, frame_idx] > 0)

        # Separate edge pixels by masks
        edge_pixels_fg = edge_pixels[foreground_mask[edge_pixels[:, 0], edge_pixels[:, 1]] > 0]
        edge_pixels_bg = edge_pixels[background_mask[edge_pixels[:, 0], edge_pixels[:, 1]] > 0]

        # Back-project edge pixels onto the horizontal and vertical planes
        points_h = back_project_to_plane(plane_h, edge_pixels_fg, calMatrix, distCoeffs)
        points_v = back_project_to_plane(plane_v, edge_pixels_bg, calMatrix, distCoeffs)

        # Compute the best shadow plane
        shadow_plane = compute_best_shadow_plane(points_h, points_v)

        # Add the computed shadow plane to the list
        shadow_planes.append(shadow_plane)

    return shadow_planes

# Usage example:
# Assuming calMatrix, distCoeffs, foreground_mask, background_mask, and front_edge_times are already defined
# plane_h, plane_v = computeTwoPlanes(calMatrix, distCoeffs, 'empty_boards.jpg')
# shadow_planes = computeShadowPlanes(foreground_mask, background_mask, front_edge_times, plane_h, plane_v, calMatrix, distCoeffs)

THIS IS NOT MY CODE, FROM ANOTHER GROUP TO COMPARE:

In [ ]:
def computeShadowPlanes(foreground_mask, background_mask, front_edge_times, plane_h, plane_v, calMatrix, distCoeffs):

    def intersect_ray_plane(plane, impts, calMatrix, distCoeffs):
        intersections = np.zeros((impts.shape[0], 3))
        a, b, c, d = plane
        impts_undistorted = cv2.undistortPoints(np.expand_dims(impts, axis=1), calMatrix, distCoeffs, None, calMatrix)
        rays = np.hstack((impts_undistorted.squeeze(), np.ones((impts_undistorted.shape[0], 1))))

        # Intersect back-projected rays and detected planes
        for i, ray in enumerate(rays):
            direction = np.linalg.solve(calMatrix, ray)
            t = -d / (a * direction[0] + b * direction[1] + c * direction[2])
            intersections[i] = t * direction
        return intersections

    def fit_plane_to_points(points):
        # Center the points
        centroid = np.mean(points, axis=0)
        centered_points = points - centroid

        # Perform SVD on the centered points
        u, s, vh = np.linalg.svd(centered_points, full_matrices=True)
        normal = vh[-1, :]
        d = -np.dot(normal, centroid)
        return np.append(normal, d)

    shadow_planes = []
    video_frames = front_edge_times.shape[2]

    for frame_number in range(video_frames):
        # Get edge pixels on the 2 planes
        foreground_pixels = np.argwhere((front_edge_times[:, :, frame_number] > 0) & foreground_mask)
        background_pixels = np.argwhere((front_edge_times[:, :, frame_number] > 0) & background_mask)

        # Compute intersection points for points on h and v
        intersections_h = intersect_ray_plane(plane_h, foreground_pixels, calMatrix, distCoeffs)
        intersections_v = intersect_ray_plane(plane_v, background_pixels, calMatrix, distCoeffs)

        # Fit a plane to the points
        all_intersections = np.vstack([intersections_h, intersections_v])
        shadow_plane = fit_plane_to_points(all_intersections)

        shadow_planes.append(shadow_plane)

    return shadow_planes